## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [16]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
       
        input_size = 28 * 28 
        hidden_size = 100 
        output_size = 10  
       
        self.W1 = tf.Variable(tf.random.normal([input_size, hidden_size]))
        self.b1 = tf.Variable(tf.zeros([hidden_size]))
        self.W2 = tf.Variable(tf.random.normal([hidden_size, output_size]))
        self.b2 = tf.Variable(tf.zeros([output_size]))
        

    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        # 实现模型函数体，返回未归一化的logits
        x = tf.reshape(x, [-1, 28*28])
        h1 = tf.nn.relu(tf.matmul(x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [17]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [19]:
train_data, test_data = mnist_dataset()
for epoch in range(500):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 28.39385 ; accuracy 0.2963
epoch 1 : loss 28.079199 ; accuracy 0.29945
epoch 2 : loss 27.770891 ; accuracy 0.30266666
epoch 3 : loss 27.468822 ; accuracy 0.30625
epoch 4 : loss 27.172737 ; accuracy 0.30963334
epoch 5 : loss 26.882496 ; accuracy 0.31293333
epoch 6 : loss 26.597946 ; accuracy 0.31595
epoch 7 : loss 26.318974 ; accuracy 0.31945
epoch 8 : loss 26.045462 ; accuracy 0.32251668
epoch 9 : loss 25.777317 ; accuracy 0.32561666
epoch 10 : loss 25.514442 ; accuracy 0.32901666
epoch 11 : loss 25.256699 ; accuracy 0.33206666
epoch 12 : loss 25.003965 ; accuracy 0.33511665
epoch 13 : loss 24.756102 ; accuracy 0.33793333
epoch 14 : loss 24.512829 ; accuracy 0.34083334
epoch 15 : loss 24.273989 ; accuracy 0.3436
epoch 16 : loss 24.039467 ; accuracy 0.34635
epoch 17 : loss 23.809223 ; accuracy 0.3496
epoch 18 : loss 23.583185 ; accuracy 0.35283333
epoch 19 : loss 23.361296 ; accuracy 0.35548332
epoch 20 : loss 23.143438 ; accuracy 0.35848334
epoch 21 : loss 22.929543 ; ac